[![Open In Colab](images/colab-badge.svg)](https://colab.research.google.com/github/BRomans/UnicornPythonEssentialsToolkit/blob/main/notebooks/P300_Epoching.ipynb)



In [ ]:
!pip install mne # Use only on Google Colab

# Day 1 - Introduction to BCI
The following notebook has been developed by Michele Romani, PhD student at the University of Trento. The notebook is part of his research on Brain-Computer Interfaces for Gaming using event-related potentials. The notebook can be used for educational purposes together with the dataset available in the [repository](https://github.com/BRomans/WorkshopBCI2025).
Copying and distribution of the notebook or part of its content must include the original author and the link to the repository.

_University of Trento, 2025_

__©Michele Romani__

## EEG Data Exploration and Preprocessing
In this notebook, we will explore and preprocess EEG data collected using a consumer EEG device. We will use the MNE library for EEG data analysis.
First, make sure that the device is connected and that you have configured a python environment with the necessary libraries installed, including MNE.
If you don't know how to do it, check the _README.md_ file in the repository.


## NeuroPawn Knight
The NeuroPawn Knight is a versatile EEG board that can be used for various applications, including brain-computer interfaces and neurofeedback.
The device features 8 configurable EEG channels, a sampling rate of 125 Hz, and supports real-time data streaming via serial communication.
In this section, we will set up the board using [Brainflow](https://brainflow.org/) and start streaming data from it as recommended in the official [documentation](https://github.com/NeuroPawn/brainflow).

In [11]:
import time

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

class KnightBoard:
    def __init__(self, serial_port: str, num_channels: int):
        """Initialize and configure the Knight Board."""
        self.params = BrainFlowInputParams()
        self.params.serial_port = serial_port
        self.num_channels = num_channels

        # Initialize board
        self.board_shim = BoardShim(BoardIds.NEUROPAWN_KNIGHT_BOARD.value, self.params)
        self.board_id = self.board_shim.get_board_id()
        self.eeg_channels = self.board_shim.get_exg_channels(self.board_id)
        self.sampling_rate = self.board_shim.get_sampling_rate(self.board_id)

    def start_stream(self, buffer_size: int = 450000):
        """Start the data stream from the board."""
        self.board_shim.prepare_session()
        self.board_shim.start_stream(buffer_size)
        print("Stream started.")
        time.sleep(2)
        for x in range(1, self.num_channels + 1):
            time.sleep(0.5)
            cmd = f"chon_{x}_12"
            self.board_shim.config_board(cmd)
            print(f"sending {cmd}")
            time.sleep(1)
            rld = f"rldadd_{x}"
            self.board_shim.config_board(rld)
            print(f"sending {rld}")
            time.sleep(0.5)

    def stop_stream(self):
        """Stop the data stream and release resources."""
        self.board_shim.stop_stream()
        self.board_shim.release_session()
        print("Stream stopped and session released.")

## Data Acquisition
Good! Now that we have defined the `KnightBoard` class, we can use it to start streaming data from the NeuroPawn Knight board.
We need to specify the serial port where the board is connected and the number of EEG channels we want to use.
If you are on Windows, the serial port will be something like `COM3`, while on Linux or MacOS it will be something like `/dev/ttyUSB0` or `/dev/ttyACM0`.

In [12]:
serial_port = "COM3"  # Change this to your serial port
num_channels = 8      # Number of EEG channels
session_duration_s = 10  # Duration of the session in seconds
board_id = BoardIds.NEUROPAWN_KNIGHT_BOARD

Knight_board = KnightBoard("COM3", 8)
Knight_board.start_stream()
eeg_channels = BoardShim.get_eeg_channels(board_id.value)

time.sleep(session_duration_s) # Let the board stream data for the specified duration
data = Knight_board.board_shim.get_board_data()

Knight_board.stop_stream()

eeg_channels

Stream started.
sending chon_1_12
sending rldadd_1
sending chon_2_12
sending rldadd_2
sending chon_3_12
sending rldadd_3
sending chon_4_12
sending rldadd_4
sending chon_5_12
sending rldadd_5
sending chon_6_12
sending rldadd_6
sending chon_7_12
sending rldadd_7
sending chon_8_12
sending rldadd_8
Stream stopped and session released.


[1, 2, 3, 4, 5, 6, 7, 8]

Very good! let's have a quick look at the structure of the data we have acquired.
We transpose the data to have channels in rows and time points in columns.
As you can see, we have 8 channels and a number of time points equal to the sampling rate (125 Hz) multiplied by the duration of the session (10 seconds).
This data is raw and unprocessed, so we will need to preprocess it before using it for any analysis.

In [13]:
import pandas as pd


eeg_data = data[eeg_channels, :]

eeg_data.T

array([[    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       ...,
       [-1302.08349609, -1302.08349609, -1302.08349609, ...,
        -1259.44628906, -1160.5423584 , -1083.4934082 ],
       [  317.77227783,   345.26986694,   343.64068604, ...,
          561.3963623 ,   593.94049072,   550.23040771],
       [ 1172.62219238,  1173.29772949,  1175.64221191, ...,
          859.77722168,   608.1661377 ,   200.27163696]])

## Converting the data to MNE format
Now that we have acquired the data, we need to convert it to a format that can be used by the [MNE](https://mne.tools/stable/index.html) library.
MNE is a powerful library for EEG and MEG data analysis in Python and provides many tools for preprocessing, visualization, and analysis.
We will create an `mne.Info` object to store metadata about the EEG data, such as channel names and sampling rate.
Then, we will create an `mne.RawArray` object to store the EEG data itself.

In [14]:
from datetime import datetime, timezone
import mne

# Creating MNE objects from brainflow data arrays
ch_types = ['eeg'] * len(eeg_channels)
ch_names = ["Ch" + str(i + 1) for i in range(len(eeg_channels))]
sfreq = BoardShim.get_sampling_rate(board_id.value)
measurement_date = datetime.now(timezone.utc)
experimenter = "Michele Romani"
participant = "Test Subject"

# rescale the data to microvolts

info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_meas_date(measurement_date)
info['experimenter'] = experimenter
info['subject_info'] = {'id': participant}



raw = mne.io.RawArray(eeg_data, info)
raw

Creating RawArray with float64 data, n_channels=8, n_times=1349
    Range : 0 ... 1348 =      0.000 ...    10.784 secs
Ready.


<RawArray | 8 x 1349 (10.8 s), ~98 kB, data loaded>

# Visualizing the raw data
Now that we have created the MNE `Raw` object, we can visualize the raw EEG data using the built-in plotting functions in MNE.
This will allow us to inspect the data for any artifacts or noise that may need to be removed during preprocessing.
If we set matplotlib to interactive mode, we can scroll through the data using the mouse wheel.

In [5]:
import matplotlib.pyplot as plt

%matplotlib inline
%matplotlib qt

raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1920x1129 with 4 Axes>

Channels marked as bad:
none


## Plot the Power Spectral Density (PSD)
We can also plot the Power Spectral Density (PSD) of the raw EEG data to inspect the frequency content of the signals.
This can help us identify any noise or artifacts that may be present in the data.
The PSD is a representation of the power of the signal as a function of frequency.
It is often used in EEG analysis to identify characteristic frequency bands, such as alpha, beta, and gamma and other features that can be fed to
a machine learning algorithm.

In [6]:
import os

raw.compute_psd().plot(average=True)
folder = 'analysis'
if not os.path.exists(folder):
    os.makedirs(folder)
plt.savefig(os.path.join(folder, 'psd_raw.png'))

Effective window size : 10.608 (s)


## Filtering the data
Woah! What was that spike at 50 Hz? It looks like we have some power line noise in our data.
To remove this noise, we can apply a notch filter at 50 Hz.
Additionally, we can apply a bandpass filter to keep only the frequencies of interest, typically between 1 Hz and 30 Hz for EEG data.
Filtering the data will help us improve the signal-to-noise ratio and make it easier to analyze the EEG signals.

In [7]:
filtered_data = raw.copy().notch_filter(freqs=50.0) # Change to 60.0 if you are in the US
filtered_data.plot()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 s)



<MNEBrowseFigure size 1920x1129 with 4 Axes>

Channels marked as bad:
none


In [8]:
filtered_data.compute_psd().plot(average=True)

Effective window size : 10.608 (s)


<MNELineFigure size 1000x350 with 1 Axes>

In [9]:
# Bandpass filter between 1 and 30 Hz, change as needed but keep in mind the Nyquist frequency limit = sampling_rate / 2
filtered_data = filtered_data.filter(l_freq=1.0, h_freq=30.0)
filtered_data.compute_psd().plot(average=True)
plt.savefig(os.path.join(folder, 'psd_filtered.png'))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (3.304 s)

Effective window size : 10.608 (s)


In [17]:
# Save raw data to csv
raw_df = pd.DataFrame(raw.get_data().T, columns=ch_names)
raw_df.to_csv(os.path.join(folder, 'raw_data.csv'), index=False)

# Read data from csv
loaded_raw_df = pd.read_csv(os.path.join(folder, 'raw_data.csv'))
loaded_raw_df

,Ch1,Ch2,Ch3,Ch4,Ch5,Ch6,Ch7,Ch8
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1344,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496
1345,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1136.382568,-794.927368
1346,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1302.083496,-1259.446289,-1160.542358,-1083.493408
1347,317.772278,345.269867,343.640686,455.141113,419.298859,561.396362,593.940491,550.230408
